# Pipeline de Treinamento DNN para Detecção de Texto
## (AI vs Human) - Multi Dataset

In [2]:
import pandas as pd
import os
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, confusion_matrix
from nn import NeuralNetwork  # Importação correta do modelo DNN

## Função para Carregar Datasets

In [3]:
def load_dataset(dataset_version):
    base_path = f'../../datasets/{dataset_version}/'
    train_file = f'Dataset{dataset_version}_train_clean.csv'
    test_file = f'Dataset{dataset_version}_test_clean.csv'
    
    train = pd.read_csv(os.path.join(base_path, train_file), sep=',')
    test = pd.read_csv(os.path.join(base_path, test_file), sep=',')
    
    return train, test

## Pré-processamento com TF-IDF (Igual ao Original)

In [4]:
def tfidf_preprocessing(train, test):
    tfidf = TfidfVectorizer(max_features=1000, stop_words='english')
    X_train = tfidf.fit_transform(train['text']).toarray()
    X_test = tfidf.transform(test['text']).toarray()
    
    y_train = train['Label'].map({'AI':1, 'Human':0}).values
    y_test = test['Label'].map({'AI':1, 'Human':0}).values
    
    return X_train, y_train, X_test, y_test, tfidf

## Função de Treinamento da DNN (Ajustada)

In [5]:
def train_evaluate_dnn(X_train, y_train, X_test, y_test, input_size):
    print("\nIniciando treinamento da DNN...")
    
    # Hiperparâmetros do código original
    nn = NeuralNetwork(
        input_size=input_size,
        hidden_size=64,
        output_size=1,
        lr=1,
        dropout_rate=0.2
    )
    
    nn.train(X_train, y_train, epochs=100)
    
    # Avaliação
    preds = nn.predict(X_test)
    print("\nRelatório de Classificação:")
    print(classification_report(y_test, preds))
    print("Matriz de Confusão:")
    print(confusion_matrix(y_test, preds))
    
    return nn

## Pipeline Principal (Corrigido)

In [6]:
# Configurações
DATASET_VERSION = 4  # Altere para 3, 5, 6, etc.
SAVE_DIR = 'modelos_dnn/'
os.makedirs(SAVE_DIR, exist_ok=True)

# Passo 1: Carregar dados
train, test = load_dataset(DATASET_VERSION)

# Passo 2: Pré-processamento
X_train, y_train, X_test, y_test, tfidf = tfidf_preprocessing(train, test)

# Passo 3: Treinar DNN
dnn_model = train_evaluate_dnn(X_train, y_train, X_test, y_test, input_size=X_train.shape[1])

# Passo 4: Salvar modelo
with open(f'{SAVE_DIR}dnn_dataset{DATASET_VERSION}.pkl', 'wb') as f:
    pickle.dump(dnn_model, f)
with open(f'{SAVE_DIR}tfidf_dataset{DATASET_VERSION}.pkl', 'wb') as f:
    pickle.dump(tfidf, f)

print("\nPipeline concluído! Modelos salvos em 'modelos_dnn/'.")


Iniciando treinamento da DNN...
Epoch 0: Loss = 0.6931465786714627, Accuracy = 49.97%
Epoch 99: Loss = 0.6928772436053634, Accuracy = 63.76%

Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.63      0.70      0.66     22500
           1       0.66      0.58      0.62     22500

    accuracy                           0.64     45000
   macro avg       0.64      0.64      0.64     45000
weighted avg       0.64      0.64      0.64     45000

Matriz de Confusão:
[[15734  6766]
 [ 9433 13067]]

Pipeline concluído! Modelos salvos em 'modelos_dnn/'.
